In [1]:
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')
tf.config.set_visible_devices([], 'GPU')

In [2]:
df = pd.read_csv("data/modified_0419.csv")
df.shape

(1131682, 92)

In [3]:
import pandas as pd
import numpy as np
from typing import Union, List
'''
도은수정 - 0420 버전
- 결측 처리 완료
- 로그변환 추가
- 다중공선성 : fico_avg 변수 추가
'''
class Preprocessor0420:
    def __init__(self):
        self.file_path = ""
        self.folder_path = ""
        self.df = pd.DataFrame()

    def __init__(self, data_file_path:str="", folder_path:str=""):
        self.file_path = data_file_path
        self.folder_path = folder_path
        self.df = pd.DataFrame()

    def load_origin_file(self, file_path:str)->None:
        if file_path!="":
            self.file_path = file_path
        self.df = pd.read_csv(self.file_path)

    def load_modified_file(self, file_path:str)->None:
        if file_path!="":
            self.file_path = file_path
        self.df = pd.read_csv(self.file_path)
        if 'Unnamed: 0' in self.df.columns:
            self.df.drop(columns=['Unnamed: 0'], inplace=True)

    def drop_columns(self, drop_columns_file_path:str)->None:
        return None

    def __preprocess_target_variable(self, target_variable:str="loan_status")->None:
        return None

    def __fill_na_with_value(self, columns:List[str], filling_value:Union[str, int])->None:
        '''
        df: dataframe to fill NA
        column_name : column name to change NA values
        filling_value : value type or just value to fill column's NA
        '''
        # for column_name in columns:
        #     if filling_value=="mode":
        #         mode_value = self.df[column_name].mode()[0]
        #     elif filling_value=="median":
        #         mode_value = self.df[column_name].median()
        #     else:
        #         mode_value = filling_value
        #     self.df[column_name].fillna(mode_value, inplace=True)
        return None

    def __preprocessing_na(self)->None:
        ## 결측 처리
        # 결측 개수가 1천 건 이하인 경우는 해당 데이터(row) 삭제
        # A1. 최빈값 대체
        # A2. 중앙값 대체
        # B. 2015년 대체
        # C. 2012년 대체
        # D. 결측 0 대체
        return None

    def __Multicollinearity(self)->None:
        # self.df.drop(columns=['fico_range_low'], inplace=True)
        self.df['fico_avg'] = (self.df['fico_range_low'] + self.df['fico_range_high'])/2
        self.df.drop(columns=['fico_range_low', 'fico_range_high'], inplace=True)

    def __convert_log(self, term):
        return np.log(term + 1)

    def __log_transform(self):
        variables = [
          "all_util", "annual_inc", "annual_inc_joint", "bc_open_to_buy", # "avg_cur_bal",
          "delinq_amnt", "dti", "max_bal_bc", "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op",
          "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc", "mths_since_rcnt_il",
          "mths_since_recent_bc", "num_accts_ever_120_pd", "num_actv_bc_tl", "num_actv_rev_tl",
          "num_bc_sats", "num_bc_tl", "num_il_tl", "num_op_rev_tl", "num_rev_accts",
          "num_rev_tl_bal_gt_0", "num_sats", "open_acc", "open_acc_6m", "open_act_il",
          "open_il_12m", "open_il_24m", "open_rv_12m", "open_rv_24m", "pub_rec_bankruptcies",
          "revol_bal", "revol_bal_joint", "tax_liens", "tot_cur_bal", "tot_hi_cred_lim",
          "total_acc", "total_bal_ex_mort", "total_bal_il", "total_bc_limit", "total_cu_tl",
          "total_il_high_credit_limit", "total_rev_hi_lim"
        ]

        # 각 변수에 대해 로그 변환 수행 및 기존 변수 삭제
        for var in variables:
            # 로그 변환 후 변수 이름에 '_log'를 추가하여 새로운 변수 생성
            new_var = var + "_log"
            # 해당 변수가 0보다 큰 경우에만 로그 변환 수행하여 음수 무한대를 방지
            # 로그 변환 후에는 기존 값이 0인 경우 음수 무한대로 처리되므로 이에 대한 처리도 필요
            # 여기서 로그는 자연로그 (밑이 e인 로그)
            self.df[new_var] = np.log(self.df[var] + 1)  # 0이 아닌 값이어야 하므로 +1 추가
            # 기존 변수 삭제
            self.df.drop(columns=[var], inplace=True)
        return self.df

    def __convert_object_to_numeric(self, column_name:str)->pd.DataFrame:
        return None

    def __convert_object_to_one_hot(self, column_name:str)->None:
        return None

    def __preprocessing_objects(self)->None:
        return None

    def preprocess(self)->None:
        # loan_status 제외 모든 column이 결측치(na)인 행 제거 (1개 행 제거됨)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)
        self.__preprocess_target_variable()
        # 결측치 제거
        self.__preprocessing_na()
        ## object 처리하기
        self.__preprocessing_objects()
        ## 다중공선성 제거 - 0419 추가
        self.__Multicollinearity()
        ## 로그변환 - 0419 추가
        # self.__log_transform()
        # index 재설정
        self.df.reset_index(drop=True, inplace=True)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)

    def get_df(self)->pd.DataFrame:
        return self.df

In [4]:
p = Preprocessor0420()
# lending_club_2020_train.csv 파일이 있는 절대 경로 혹은 상대 경로를 명시해주세요
p.load_modified_file(file_path="data/modified_0419.csv")
# preprocess를 돌리기
p.preprocess()
df = p.get_df()
df.shape

(1131682, 90)

In [5]:
df.to_csv("data/modified_0420_non_log.csv")